Your dataset: The folder 'sphaera_illustrations_dataset' contains 844 illustrations, selected from all illustrations that have been extracted from the books in the sphaera corpus 

### 1. Cryptographic hashing for duplicate detection

Datasets are never clean. In image datasets it might be a good idea to check for (exact) duplicates before starting to analyze the data.

Cryptographic hashing techniques produce a fixed size fingerprint of an inputfile in such a way, that small changes in the file result in large changes in the hash. To find identical files, we can thus compare the hashes instead of the files them selfs, which due to the small size of the hashes is much faster.

#### Task:

We have hidden on duplicate file in the datatset. To find it:

* hash all files in the dataset with the md5 hash function (use the hashlib library)
* compare the hashes to find the duplicate


In [ ]:
import hashlib
import os

dataset_dir = '/home/space/datasets/workshop-dh-ml/image-similarity/sphaera_illustrations_dataset' 
hash_to_filename = {}

for filename in os.listdir(dataset_dir):
    
    if filename == '.DS_Store':
        pass # This is for Mac users. Do nothing if your folder contains the notorious .DS_Store file
        
    else:
        filepath = os.path.join(dataset_dir, filename)
        filehash = hashlib.md5(open(filepath,  'rb').read()).hexdigest()
        
        if filehash in hash_to_filename.keys():
            print('We found a duplicate pair: {} and {}'.format(filename, hash_to_filename[filehash]))
        else: 
            hash_to_filename[filehash] = filename

### 2. Hash an image

In image hashing or fingerprinting, opposed to cryptographic hashing we want to produce fingerprints which are not very sensitive to little changes such that similar images produce similar hashes.

#### Task:

To hash an image: 

* Read in "2106_sacrobosco_sphaera_1561_p25_348,1242,509,433.jpg" with PIL. Have a look at the image.
* Convert to grayscale
* Scale to 8x 8 pixel
* Compute the mean pixe intensity (hint: use numpy)
* Convert every pixel with less than mean intensity to black and every pixel grater equal than than mean intensity to white
* Visualize result
* Flatten the resulting image

If you succeed you have created your first image hash. This particular has function is called average hash

In [ ]:
from PIL import Image, ImageOps
import numpy as np
from matplotlib import pyplot as plt


def get_avarage_hash(filename):
    
    filepath = os.path.join(dataset_dir, filename)
    img = Image.open(filepath)
    img2 = ImageOps.grayscale(img)
    img3 = img2.resize((8,8))

    avarage_hash = np.where(img3 > np.array(img3).mean(), 1, 0)

    return img, avarage_hash


img_1, avarage_hash_1 = get_avarage_hash("1701_sacrobosco_sphaera_1550_p40_395,1295,501,430.jpg")

fig, (ax1, ax2) = plt.subplots(1, 2)
ax1.imshow(img_1, cmap='gray')
ax2.imshow(avarage_hash_1, cmap='gray')
plt.show()

print('Here is our first image hash: {}'.format(avarage_hash_1.flatten()))



### 3. Compare  images via their hashes

#### Task:

* Calculate and visualize the average hash for "2106_sacrobosco_sphaera_1561_p25_348,1242,509,433.jpg" 
* Compare it to the hash calculated before, visualize the pixels where the two hashes are different.



In [ ]:
img_2, avarage_hash_2 = get_avarage_hash("2106_sacrobosco_sphaera_1561_p25_348,1242,509,433.jpg")

fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2)
ax1.imshow(img_1, cmap='gray')
ax2.imshow(avarage_hash_1, cmap='gray')
ax3.imshow(img_2, cmap='gray')
ax4.imshow(avarage_hash_2, cmap='gray')
plt.show()

diff = avarage_hash_1 == avarage_hash_2
plt.imshow(diff, cmap="Blues_r")
plt.show()

With the average hash we have indeed engineered a function that has  mapped our two _visually_ similar imaged onto two representation that are indeed _bitwise_ similar

### 4. Use image hashing to find  images in the corpus that are similar to a given image.

#### Task:

Find all images in the corpus whose average hashes differ from that of our original image ("2106_sacrobosco_sphaera_1561_p25_348,1242,509,433.jpg") by no more than 10 bit.

In [ ]:
import re

for filename in os.listdir(dataset_dir):
    
    if filename == '.DStore':
        pass
    
    else:
        img , avarage_hash = get_avarage_hash(filename)
        hamming_dist = np.count_nonzero(avarage_hash_1.flatten() != avarage_hash.flatten())
        
        if hamming_dist  <= 10:
            
            print(re.sub(r'_\d*,.*', '', filename))
            filepath = os.path.join(dataset_dir, filename)
            img = Image.open(filepath)
            plt.imshow(img, cmap='gray')
            plt.show()

This is the basic idea of finding similar images by means of image hashing. Of course there is a lot we can do to improve. In our case for instance we are only interested in what was actually printed, and not in the background information such as paper color or texture. We should thus for instance convert to black and white before comparing.

Also there are image hashing functions such as the perceptual hash, which, depending on the nature of your data, may be better suited. Of course there is a library for everything in Python, so you do not have to implement them yourself. Have a look at: https://pypi.org/project/ImageHash/ 

### 5. Limits of image hashing 

Image hashing is a quick solution for retrieving similar images, but it is also rather crude with some severe limitation. Thus it is not invariant against translations or rotations of the images, which however, is often desirable when searching for similar imaged. It its also for instance not sensitive to the similarity between and image and parts thereof, for instance different crops.

#### Task:
* take our original image ("2106_sacrobosco_
sphaera_1561_p25_348,1242,509,433.jpg" ) and cut of a quarter of its width on the right
* calculate the average hash and compare to the average hash of the original



In [ ]:
new_width = int(np.array(img_1).shape[1] - np.array(img_1).shape[1]/4)
img_cut = np.array(img_1)[:,:new_width]

img2 = ImageOps.grayscale(Image.fromarray(img_cut))
img3 = img2.resize((8,8))
avarage_hash_4 = np.where(img3 > np.array(img3).mean(), 1, 0)

fig, ((ax1, ax2),(ax3, ax4)) = plt.subplots(2, 2)
ax1.imshow(img_cut, cmap='gray')
ax2.imshow(avarage_hash_4, cmap='gray')
ax3.imshow(img_1, cmap='gray')
ax4.imshow(avarage_hash_2, cmap='gray')
plt.show()

diff = avarage_hash_1 == avarage_hash_4
plt.imshow(diff, cmap="Blues_r")
plt.show()

hamming_dist = np.count_nonzero(avarage_hash_1.flatten() != avarage_hash_4.flatten())
print('The original and the cropped image differ by {} bits.'.format(hamming_dist))

#### Additional task: 
* Go back to 4. and find all images whose avarage hashes differ from that of our original image by 16 or less bits

# Keypoint/feature detection and matching

### Fingerprint matching as a blueprint for the approach
|  |  |  |
|---|---|---|
||<img src=fingerprint.jpg>||
|&nbsp;|<small>Srinivasan, Harish & J. Beal, Matthew & Srihari, Sargur. (2005). Machine learning approaches for person identification and verification. Proceedings of SPIE - The International Society for Optical Engineering. 5778.</small>|&nbsp;|

* find robust features in finger ridge patterns (minutae, e.g. ridge endings or bifurcations) 
* match features
* count the number of matches

> ... the probability that a fingerprint with 36 minutiae points will share 12 minutiae points with another arbitrarily chosen fingerprint with 36 minutiae points is 6.10 × 10<sup>−8</sup>.

 
<small>Sharath Pankanti, Salil Prabhakar, and Anil K. Jain. 2002. On the Individuality of Fingerprints. IEEE Trans. Pattern Anal. Mach. Intell. 24, 8 (August 2002), 1010-1025. </small>


### 1. Keypoint extraction

#### Task:

* Read in "1852_sacrobosco_commentaria_1578_p215_498,157,604,604.jpg", find robust keypoints and visualize some of them

A number of algorithms for local feature detection and description are available, e.g. SURF (speeded up robust features) and SIFT (scale-invariant feature transform). ORB (Oriented FAST and rotated BRIEF) is a free alternative, part of the openCV computer vision library. A good introduction to keypoint extraction and matching can be found here: https://docs.opencv.org/master/db/d27/tutorial_py_table_of_contents_feature2d.html

In [ ]:
import cv2

orb = cv2.ORB_create()
filepath = os.path.join(dataset_dir, '1852_sacrobosco_commentaria_1578_p215_498,157,604,604.jpg')
img_1 = cv2.imread(filepath)

kp1, des1 = orb.detectAndCompute(img_1,None)
pts = np.array([kp1[i].pt for i in range(0, len(kp1))]).reshape(-1, 2)
plt.figure(figsize=(8,8))
plt.scatter(pts[:40,0],pts[:40,1], marker='o', facecolors='none', edgecolors='r')
plt.imshow(img_1) 
plt.show()

#### Task:

* find keypoints and descriptors for a second image "2203_giuntini_commentaria_1577_p219_523,108,502,507.jpg"
* match the keypoints between the two images (use BFMatcher https://docs.opencv.org/4.5.2/d3/da1/classcv_1_1BFMatcher.html)
* visualize the 20 best matches

In [ ]:
filepath = os.path.join(dataset_dir, '2203_giuntini_commentaria_1577_p219_523,108,502,507.jpg')
img_2 = cv2.imread(filepath)

kp2, des2 = orb.detectAndCompute(img_2,None)

bf = cv2.BFMatcher(cv2.NORM_HAMMING , crossCheck=True)
matches = bf.match(des1,des2)
matches = np.array(sorted(matches, key = lambda x:x.distance))
img_matches = cv2.drawMatches(img_1,kp1,img_2,kp2,matches[0:100], None, matchColor = (255,0,0),flags=2)
plt.figure(figsize=(10,10))
plt.imshow(img_matches)
plt.show()

There are a number of ways to go from here. A simple one is to count the number of good matches between to images of a certain quality.

#### Task:

* match the features of "1959_vinet_sphaera_1594_p112_119,82,560,695.jpg" as a reference file to the features of every other image in your dataset
* count the number of good matches (match.distance <50)
* visualize the images that have at least 50 good keypoint matches with the reference file

In [ ]:
filepath = os.path.join(dataset_dir, '1959_vinet_sphaera_1594_p112_119,82,560,695.jpg')
img_1 = cv2.imread(filepath)

kp1, des1 = orb.detectAndCompute(img_1,None)


for j, filename in enumerate(os.listdir(dataset_dir)):
    
    if filename == '.DStore':
        pass
    
    else:
        #print(j, end='\r')
        filepath = os.path.join(dataset_dir, filename)
        img_2 = cv2.imread(filepath)

        kp2, des2 = orb.detectAndCompute(img_2,None)
        matches, good = bf.match(des1,des2), []
    
        count = 0
    
        for m in matches:
            if m.distance < 50:
                good.append(m)
                
        
                
        if len(good) > 50:
            print(filepath)
            #img = Image.open(filepath)
            count += 1
            plt.imshow(img_2)
            plt.show()
            
count

In [ ]:
# A slightly better version with FLANN matcher

filepath = os.path.join(dataset_dir, '1959_vinet_sphaera_1594_p112_119,82,560,695.jpg')
img_1 = cv2.imread(filepath)

kp1, des1 = orb.detectAndCompute(img_1,None)


for j, filename in enumerate(os.listdir(dataset_dir)):
    
    if filename == '.DStore':
        pass
    
    else:
        filepath = os.path.join(dataset_dir, filename)
        img_2 = cv2.imread(filepath)

        kp2, des2 = orb.detectAndCompute(img_2,None)
        index_params= dict(algorithm = 6,
                           table_number = 6, # 12
                           key_size = 12,     # 20
                           multi_probe_level = 1) #2

        search_params = dict(checks = 50)
        flann = cv2.FlannBasedMatcher(index_params, search_params)

        matches = flann.knnMatch(des1,des2,k=2)


        # Need to draw only good matches, so create a mask
        try:
            matchesMask = [[0,0] for i in range(len(matches))]
            good = []
            for i,(m,n) in enumerate(matches):
                if m.distance < 0.7*n.distance:
                    matchesMask[i]=[1,0]
                    good.append([m])
                    
            draw_params = dict(matchColor = (0,255,0),
                               singlePointColor = (255,0,0),
                               matchesMask = matchesMask,
                               flags = cv2.DrawMatchesFlags_DEFAULT)


            if len(good) > 16:
                img3 = cv2.drawMatchesKnn(img_1,kp1,img_2,kp2,matches,None,**draw_params)
                plt.imshow(img3,)
                plt.show()
                print('similar to {}'.format(filename))
        except:
            pass


# CNN embedding distances

## 1. Get Embedding

During training CNNs learn to build up rich representations of the input images. This can be exploited for detecting similarity by cutting a pretrained CNN at a certain layer and calculating the distance between the embeddings of two images at that layer. The smaller the distance the higher (hopefully) the similarity. The deeper (further away from input) one moves into the network  the more complex and abstract and the less local  the features that make up these representations (embeddings) become. Thus we expect that the similarity we recover with this method moves from from concrete to more abstract if we use embeddings produced deeper into the network.

#### Task:

* cut a VGG16 pretrained on image net after the fourth convolutional block, i.e at layer 13 
* calculate the embeddings for all images in the dataset

In [ ]:
! pip install keras

In [ ]:
import tensorflow as tf
from keras.applications.vgg16 import VGG16
from keras import backend as K
from keras.layers import merge, Input
from keras.preprocessing import image



cut_layer = 14


image_input = Input(shape=(224, 224, 3))
model = VGG16(include_top=True, weights="imagenet",
              input_tensor=image_input)  

model.summary()

print('Will cut at layer {}'.format(model.layers[cut_layer].output))

inp = model.input  # input placeholder
outputs = model.layers[cut_layer].output
print(inp)
print(outputs)
functor = K.function([inp], [outputs])  # evaluation function

layer_outs_arr = []

for j, filename in enumerate(os.listdir(dataset_dir)):
    
    if filename == '.DStore':
        pass
    
    else:
        filepath = os.path.join(dataset_dir, filename)
        if (j % 100) == 0:
            print(j, filepath)

        img = image.load_img(filepath, target_size=(224, 224))
        img_data = image.img_to_array(img)
        img_data = np.expand_dims(img_data, axis=0)  # simply creates batch of size one as required by preprocess_input
        img_data = tf.keras.applications.vgg16.preprocess_input(img_data)
        layer_out = functor([img_data, 1])
        layer_out = np.array(layer_out)
        
        layer_outs_arr.append(layer_out.flatten())

layer_outs_arr = np.array(layer_outs_arr)
    
    

## 2. Clustering

#### Task:

* cluster the embeddings using k-means with  15 clusters 
* visualize some of the clusters

In [ ]:
from sklearn.cluster import KMeans, MiniBatchKMeans
from sklearn.metrics import pairwise_distances

km = KMeans(init='k-means++', n_clusters=15, 
                        verbose=1, n_init=3, max_iter=300)
clustering = km.fit(layer_outs_arr)
predictions = clustering.predict(layer_outs_arr)
predictions, predictions.shape




In [ ]:
for image in np.array(os.listdir(dataset_dir))[np.where(predictions==1)]:
    filepath = os.path.join(dataset_dir, image)
    img = cv2.imread(filepath)
    plt.imshow(img)
    plt.show()

In [ ]:


    for i in range(15):
        for image in np.array(os.listdir(dataset_dir))[np.where(predictions==i)]:
            filepath = os.path.join(dataset_dir, image)
            img = cv2.imread(filepath)
            plt.imshow(img)
            plt.show()
            break

In [ ]:

km = KMeans(init='k-means++', n_clusters=11, 
                        verbose=1, n_init=3, max_iter=300)
clustering = km.fit(layer_outs_arr)
predictions = clustering.predict(layer_outs_arr)
predictions, predictions.shape




In [ ]:
    for i in range(15):
        for ima in np.array(os.listdir(dataset_dir))[np.where(predictions==i)]:
            filepath = os.path.join(dataset_dir, ima)
            img = cv2.imread(filepath)
            plt.imshow(img)
            plt.show()
            break

In [ ]:
import tensorflow as tf
from keras.applications.vgg16 import VGG16
from keras import backend as K
from keras.layers import merge, Input
from keras.preprocessing import image
cut_layer = 20


image_input = Input(shape=(224, 224, 3))
model = VGG16(include_top=True, weights="imagenet",
              input_tensor=image_input)  

model.summary()

print('Will cut at layer {}'.format(model.layers[cut_layer].output))

inp = model.input  # input placeholder
outputs = model.layers[cut_layer].output
print(inp)
print(outputs)
functor = K.function([inp], [outputs])  # evaluation function

layer_outs_arr = []

for j, filename in enumerate(os.listdir(dataset_dir)):
    
    if filename == '.DStore':
        pass
    
    else:
        filepath = os.path.join(dataset_dir, filename)
        if (j % 100) == 0:
            print(j, filepath)

        img = image.load_img(filepath, target_size=(224, 224))
        img_data = image.img_to_array(img)
        img_data = np.expand_dims(img_data, axis=0)  # simply creates batch of size one as required by preprocess_input
        img_data = tf.keras.applications.vgg16.preprocess_input(img_data)
        layer_out = functor([img_data, 1])
        layer_out = np.array(layer_out)
        
        layer_outs_arr.append(layer_out.flatten())

layer_outs_arr = np.array(layer_outs_arr)
    
 

In [ ]:
km = KMeans(init='k-means++', n_clusters=11, 
                        verbose=1, n_init=3, max_iter=300)
clustering = km.fit(layer_outs_arr)
predictions = clustering.predict(layer_outs_arr)
predictions, predictions.shape


In [ ]:
    for i in range(15):
        for ima in np.array(os.listdir(dataset_dir))[np.where(predictions==i)]:
            filepath = os.path.join(dataset_dir, ima)
            img = cv2.imread(filepath)
            plt.imshow(img)
            plt.show()
            break

In [ ]:
import tensorflow as tf
from keras.applications.vgg16 import VGG16
from keras import backend as K
from keras.layers import merge, Input
from keras.preprocessing import image
cut_layer = 15


image_input = Input(shape=(224, 224, 3))
model = VGG16(include_top=True, weights="imagenet",
              input_tensor=image_input)  

model.summary()

print('Will cut at layer {}'.format(model.layers[cut_layer].output))

inp = model.input  # input placeholder
outputs = model.layers[cut_layer].output
print(inp)
print(outputs)
functor = K.function([inp], [outputs])  # evaluation function

layer_outs_arr = []

for j, filename in enumerate(os.listdir(dataset_dir)):
    
    if filename == '.DStore':
        pass
    
    else:
        filepath = os.path.join(dataset_dir, filename)
        if (j % 100) == 0:
            print(j, filepath)
            
        img = image.load_img(filepath, target_size=(112, 112)) # load image in every rotation pos
        stimg = np.stack
        img_data = image.img_to_array(img)
        img_data1 = np.rot90(img_data)
        img_data2 = np.rot90(img_data1)
        img_data3 = np.rot90(img_data2)
        
        stimg = np.stack((img_data, img_data1, img_data2, img_data3))
        stimg=stimg[None]
        stimg =stimg.reshape(2, 2, 112, 112, 3)
       
        stimg =stimg.transpose(0,2,1,3,4)
        stimg = stimg.reshape(224, 224, 3)
        if (j == 0):
            fig = plt.figure()
            plt.imshow((stimg * 255).astype(np.uint8))
            plt.show()
            
        img_data = np.expand_dims(stimg, axis=0)  # simply creates batch of size one as required by preprocess_input
        img_data = tf.keras.applications.vgg16.preprocess_input(img_data)
        layer_out = functor([img_data, 1])
        layer_out = np.array(layer_out)
        
        layer_outs_arr.append(layer_out.flatten())

layer_outs_arr = np.array(layer_outs_arr)
    
 

In [ ]:
from sklearn.cluster import KMeans, MiniBatchKMeans
from sklearn.metrics import pairwise_distances

km = KMeans(init='k-means++', n_clusters=10, 
                        verbose=1, n_init=3, max_iter=300)
clustering = km.fit(layer_outs_arr)
predictions = clustering.predict(layer_outs_arr)
predictions, predictions.shape


In [ ]:
figure, axis=plt.subplots(15,5, figsize= (20,20))
 
for i in range(15):
    count = 0
    for ima in np.array(os.listdir(dataset_dir))[np.where(predictions==i)]:
        
        ax=axis[i, count]
        count+=1
        filepath = os.path.join(dataset_dir, ima)
        img = cv2.imread(filepath)
        ax.imshow(img)
        #plt.show()
        if count == 5:
            break